In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time


In [ ]:
distances = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0]
energies = []

for r in distances:
  print('r=',r)
  symbols = ["Li", "H"]
  geometry = np.array([0.0, 0.0, 0.0, 0.0, 0.0, r])

  H, qubits = qchem.molecular_hamiltonian(
      symbols,
      geometry,
      active_electrons=2,
      active_orbitals=5
  )

  active_electrons = 2

  singles, doubles = qchem.excitations(active_electrons, qubits)

  hf_state = qchem.hf_state(active_electrons, qubits)
  dev = qml.device("default.qubit", wires=qubits)

  #do Pauli Tapering
  generators = qml.symmetry_generators(H)
  paulixops = qml.paulix_ops(generators, qubits)

  n_electrons = 2
  paulix_sector = qml.qchem.optimal_sector(H, generators, n_electrons)

  #H = qml.taper(H, generators, paulixops, paulix_sector)
  #H = qml.Hamiltonian(np.real(H.coeffs), H.ops)
  #qubits = qubits-2

  def circuit_1(params, excitations):
      qml.BasisState(hf_state, wires=range(qubits))

      for i, excitation in enumerate(excitations):
          if len(excitation) == 4:
              qml.DoubleExcitation(params[i], wires=excitation)
          else:
              qml.SingleExcitation(params[i], wires=excitation)
      return qml.expval(H)


  dev = qml.device("default.qubit", wires=qubits)
  cost_fn = qml.QNode(circuit_1, dev, interface="autograd")

  circuit_gradient = qml.grad(cost_fn, argnum=0)

  params = [0.0] * len(doubles)
  grads = circuit_gradient(params, excitations=doubles)


  doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-5]

  step_size = 0.05
  opt = qml.QNGOptimizer(stepsize=step_size, lam=0.001)

  params_doubles = np.zeros(len(doubles_select), requires_grad=True)

  for n in range(20):
      params_doubles = opt.step(cost_fn, params_doubles, excitations=doubles_select)

  def circuit_2(params, excitations, gates_select, params_select):
      qml.BasisState(hf_state, wires=range(qubits))

      for i, gate in enumerate(gates_select):
          if len(gate) == 4:
              qml.DoubleExcitation(params_select[i], wires=gate)
          elif len(gate) == 2:
              qml.SingleExcitation(params_select[i], wires=gate)

      for i, gate in enumerate(excitations):
          if len(gate) == 4:
              qml.DoubleExcitation(params[i], wires=gate)
          elif len(gate) == 2:
              qml.SingleExcitation(params[i], wires=gate)
      return qml.expval(H)

  cost_fn = qml.QNode(circuit_2, dev, interface="autograd")
  circuit_gradient = qml.grad(cost_fn, argnum=0)
  params = [0.0] * len(singles)

  grads = circuit_gradient(
      params,
      excitations=singles,
      gates_select=doubles_select,
      params_select=params_doubles
  )


  singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-5]


  cost_fn = qml.QNode(circuit_1, dev, interface="autograd")

  params = np.zeros(len(doubles_select + singles_select), requires_grad=True)

  gates_select = doubles_select + singles_select
  conv_tol = 1e-06

  conv_en = []

  for n in range(20):
      t1 = time.time()
      params, energy = opt.step_and_cost(cost_fn, params, excitations=gates_select)
      t2 = time.time()
      conv_en.append(energy)
      if n % 4 == 0:
        print("n = {:},  E = {:.8f} H, t = {:.2f} s".format(n, energy, t2 - t1))
      if n>=2:
        conv = conv_en[n-1] - conv_en[n-2]
        if np.abs(conv) <= conv_tol:
          break

  energies.append(energy)

print(energies)